# Data Pipeline

### Our pipeline process:
#### Extract Features:
- `Airport_Pair` which will be used for `Delay_Trend_Past_Week` to see if there was a Delay trend in the previous week
- `Same_Day_Tail_Reuse`which will be using **ONLY** the date from `dep_datetime`, as well as using `Tail_Number`
- `Previous_Flight_Delay` which will be using information from `Tail_Number`, `dep_date` which will derive from `dep_datetime`, and `CRSDepTime`
- `Turnaround_Time` will also be extracted from `CRSDepTime` subtracted by `Previous_Arrival_Time`(Which is derived from `Tail_Number`, `dep_date`, and `CRSArrTime`)
- `Slack_Time` that derives from `CRSArrTime` subtracted by `CRSElaspedTime`

#### Outlier
- `Aircraft_Age` outlier must be removed 

#### Transformations: 
- `log_distance` using the log transformation 

#### Drop 
- is_Holidays

In [37]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import time
from sklearn.model_selection import cross_val_score

from sklearn import set_config
set_config(transform_output = "pandas")
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report


In [40]:
def stratified_random_split(df: pd.DataFrame, target_column: str, test_size: float = 0.1, random_state: int = 42):
    """
    Performs a stratified random train-test split to ensure all classes in 
    'Flight_Status' are proportionally represented in both sets.

    Parameters:
    df (pd.DataFrame): The dataset containing the target variable.
    target_column (str): The column representing the classification target.
    test_size (float): The proportion of data to be used as test data.
    random_state (int): Random seed for reproducibility.

    Returns:
    tuple: (train_df, test_df) DataFrames.
    """
    train_df, test_df = train_test_split(
        df, test_size=test_size, stratify=df[target_column], random_state=random_state
    )

    print(f"Train size: {len(train_df)} samples")
    print(f"Test size: {len(test_df)} samples")

    return train_df, test_df

def fix_flight_status(df):
    """
    Fixes the Flight_Status column to combine all security-related delays into 'Security Issue'.
    
    Parameters:
    df (pd.DataFrame): The flight dataset containing 'Flight_Status'.

    Returns:
    pd.DataFrame: Updated DataFrame with 'Flight_Status' column fixed.
    """
    # Combine all security-related delays into 'Security Issue'
    df["Flight_Status"] = df["Flight_Status"].replace(
        {"Major Delay - SecurityDelay": "Security Issue", "Security Cancellation": "Security Issue", 'Medium Delay - SecurityDelay': 'Security Issue', 'Minor Delay - SecurityDelay': 'Security Issue'}
    )

    return df

In [63]:
# Setting up the DataFrame for the flight data
flight_data = pd.read_parquet("WEATHER97.parquet")

flight_data = fix_flight_status(flight_data)

# Stratify the data
train_data, test_data= stratified_random_split(flight_data, target_column="Flight_Status")



Train size: 13184010 samples
Test size: 1464890 samples


In [64]:
# Splitting the data into X and y
X_train = train_data.drop(columns=["Flight_Status"])
y_train = train_data["Flight_Status"]

### Pipeline Functions

In [43]:
# Custom progress logger
class ProgressLogger(BaseEstimator, TransformerMixin):
    """
    A transformer that logs progress through a pipeline.
    """
    
    def __init__(self, total_rows, log_interval=0.01, name='Pipeline'):
        self.total_rows = total_rows
        self.log_interval = log_interval
        self.name = name
        self.start_time = None
        self.last_log_percent = -1
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # Initialize timer if not started
        if self.start_time is None:
            self.start_time = time.time()
            print(f"{self.name} processing started on {self.total_rows:,} rows")
        
        # Calculate current progress
        current_rows = X.shape[0]
        percent_complete = current_rows / self.total_rows
        
        # Check if we need to log progress
        int_percent = int(percent_complete / self.log_interval)
        if int_percent > self.last_log_percent:
            self.last_log_percent = int_percent
            elapsed = time.time() - self.start_time
            
            # Estimate time remaining
            percent_done = percent_complete * 100
            if percent_complete > 0:
                total_est = elapsed / percent_complete
                remaining = total_est - elapsed
                time_str = f" - Est. remaining: {remaining:.1f}s"
            else:
                time_str = ""
                
            print(f"{self.name}: {percent_done:.1f}% complete ({current_rows:,}/{self.total_rows:,} rows){time_str}")
        
        return X

class DelayTrendEncoder(BaseEstimator, TransformerMixin):
    """
    Transformer that precomputes a route-based delay trend using week_of_year and top N airports.

    Parameters:
        date_col (str): Name of the datetime column
        origin_col (str): Origin airport column
        dest_col (str): Destination airport column
        status_col (str): Flight status column for delay signal
        output_col (str): Name of the new delay trend feature
        top_n_airports (int): Number of top airports to retain by frequency
    """
    def __init__(self,
                 date_col='dep_datetime',
                 origin_col='OriginAirportID',
                 dest_col='DestAirportID',
                 status_col='Flight_Status',
                 output_col='Delay_Trend',
                 top_n_airports=15):
        self.date_col = date_col
        self.origin_col = origin_col
        self.dest_col = dest_col
        self.status_col = status_col
        self.output_col = output_col
        self.top_n_airports = top_n_airports

    def fit(self, X, y=None):
        X_temp = X.copy()
        X_temp[self.date_col] = pd.to_datetime(X_temp[self.date_col])
        X_temp['week_of_year'] = X_temp[self.date_col].dt.isocalendar().week

        X_temp['Flight_Status'] = y.reset_index(drop=True)
        
        # Restrict to top N airports
        all_airports = pd.concat([X_temp[self.origin_col], X_temp[self.dest_col]])
        top_airports = all_airports.value_counts().head(self.top_n_airports).index
        X_temp = X_temp[
            X_temp[self.origin_col].isin(top_airports) &
            X_temp[self.dest_col].isin(top_airports)
        ]

        # Create delay signal
        delay_reasons = [
            'CarrierDelay', 'WeatherDelay', 'NASDelay',
            'SecurityDelay', 'LateAircraftDelay'
        ]
        X_temp['delay_signal'] = 0
        mask = X_temp['Flight_Status'].str.contains('Delay', na=False)
        delay_type = X_temp.loc[mask, 'Flight_Status'].str.split(' - ').str[-1]
        valid = delay_type.isin(delay_reasons)
        X_temp.loc[mask[mask].index[valid], 'delay_signal'] = 1

        # Group by week and airport pair
        self.trend_lookup_ = (
            X_temp.groupby(['week_of_year', self.origin_col, self.dest_col])['delay_signal']
            .mean()
            .reset_index()
            .rename(columns={'delay_signal': self.output_col})
        )

        return self

    def transform(self, X):
        X = X.copy()
        X[self.date_col] = pd.to_datetime(X[self.date_col])
        X['week_of_year'] = X[self.date_col].dt.isocalendar().week

        X = X.merge(
            self.trend_lookup_,
            on=['week_of_year', self.origin_col, self.dest_col],
            how='left'
        )

        X[self.output_col] = X[self.output_col].fillna(0)
        return X.drop(columns=['week_of_year'])

class SameDayTailReuseEncoder(BaseEstimator, TransformerMixin):
    """
    Transformer that creates a feature counting how many times
    the same tail number (aircraft) is used on the same day.

    Parameters:
        datetime_col (str): Column containing full departure datetime
        tail_col (str): Column name for tail number
        output_col (str): Name of the output column
    """

    def __init__(self,
                 datetime_col='dep_datetime',
                 tail_col='Tail_Number',
                 output_col='Same_Day_Tail_Reuse'):
        self.datetime_col = datetime_col
        self.tail_col = tail_col
        self.output_col = output_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Ensure datetime is parsed correctly
        X[self.datetime_col] = pd.to_datetime(X[self.datetime_col])

        # Extract just the date portion
        X['dep_date'] = X[self.datetime_col].dt.date

        # Count reuse of same tail number per day
        X[self.output_col] = (
            X.groupby([self.tail_col, 'dep_date'])[self.tail_col]
            .transform('count')
            .astype(float)  
        )

        return X.drop(columns=['dep_date'])

    
class TurnaroundDelayEncoder(BaseEstimator, TransformerMixin):
    """
    Transformer that computes:
    - Previous_Flight_Delay: scheduled departure time of previous flight with same tail number on same day
    - Turnaround_Time: time between previous arrival and current departure

    Parameters:
        datetime_col (str): Column with full departure datetime
        dep_time_col (str): Column with scheduled departure time (e.g. CRSDepTime)
        arr_time_col (str): Column with scheduled arrival time (e.g. CRSArrTime)
        tail_col (str): Tail number column
        output_prefix (str): Prefix to use for new feature columns
    """

    def __init__(self,
                 datetime_col='dep_datetime',
                 dep_time_col='CRSDepTime',
                 arr_time_col='CRSArrTime',
                 tail_col='Tail_Number',
                 output_prefix=''):
        self.datetime_col = datetime_col
        self.dep_time_col = dep_time_col
        self.arr_time_col = arr_time_col
        self.tail_col = tail_col
        self.output_prefix = output_prefix

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.datetime_col] = pd.to_datetime(X[self.datetime_col])
        X['dep_date'] = X[self.datetime_col].dt.date

        # Sort the values
        X = X.sort_values(by=[self.tail_col, 'dep_date', self.dep_time_col])

        prev_delay_col = self.output_prefix + 'Previous_Flight_Delay'
        turnaround_col = self.output_prefix + 'Turnaround_Time'

        # Previous scheduled departure
        X[prev_delay_col] = (
            X.groupby([self.tail_col, 'dep_date'])[self.dep_time_col]
            .shift(1)
        )

        # Previous scheduled arrival time
        X['Previous_Arrival_Time'] = (
            X.groupby([self.tail_col, 'dep_date'])[self.arr_time_col]
            .shift(1)
        )

        # Compute turnaround time using a defined method
        X[turnaround_col] = self._calculate_turnaround(X[self.dep_time_col], X['Previous_Arrival_Time'])

        # Replace missing with 0
        X[prev_delay_col] = X[prev_delay_col].fillna(0)
        X[turnaround_col] = X[turnaround_col].fillna(0)

        return X.drop(columns=['dep_date', 'Previous_Arrival_Time'])

    def _calculate_turnaround(self, current_dep, previous_arr):
        """
        Applies time difference logic with wrap-around at midnight (2400).
        """
        diff = current_dep - previous_arr
        adjusted = diff.mask((~diff.isna()) & (diff < 0), diff + 2400)
        return adjusted

class SlackTimeEncoder(BaseEstimator, TransformerMixin):
    """
    Transformer that calculates slack time as the difference between scheduled
    arrival time (CRSArrTime) and scheduled elapsed time (CRSElapsedTime).

    Parameters:
        arr_col (str): Column name for scheduled arrival time.
        elapsed_col (str): Column name for scheduled elapsed time.
        output_col (str): Name of the output column to store slack time.
    """

    def __init__(self,
                 arr_col='CRSArrTime',
                 elapsed_col='CRSElapsedTime',
                 output_col='Slack_Time'):
        self.arr_col = arr_col
        self.elapsed_col = elapsed_col
        self.output_col = output_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.output_col] = X[self.arr_col] - X[self.elapsed_col]
        return X

### Pipeline Model

In [45]:
# import random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


numerical_features = [
    'Distance', 'relative_humidity_2m', 'temperature_2m',
    'Delay_Trend', 'Same_Day_Tail_Reuse', 'Previous_Flight_Delay',
    'Turnaround_Time', 'Slack_Time',
    'Aircraft_Seats', 'Aircraft_Engines', 'Aircraft_Age', 'Is_Freighter',
    'Is_Holiday_Week',
    'temperature_2m_dest_dep_time', 'relative_humidity_2m_dest_dep_time',
    'precipitation', 'showers', 'snowfall', 'snow_depth',
    'soil_moisture_0_to_1cm', 'et0_fao_evapotranspiration', 'evapotranspiration',
    'cloud_cover_high', 'cloud_cover_low', 'surface_pressure', 'weather_code',
    'pressure_msl', 'wind_direction_10m', 'wind_gusts_10m',
    'precipitation_dest_dep_time', 'showers_dest_dep_time',
    'snowfall_dest_dep_time', 'snow_depth_dest_dep_time',
    'soil_moisture_0_to_1cm_dest_dep_time',
    'et0_fao_evapotranspiration_dest_dep_time',
    'evapotranspiration_dest_dep_time', 'cloud_cover_high_dest_dep_time',
    'cloud_cover_low_dest_dep_time', 'surface_pressure_dest_dep_time',
    'weather_code_dest_dep_time', 'pressure_msl_dest_dep_time',
    'wind_direction_10m_dest_dep_time', 'wind_gusts_10m_dest_dep_time', 'CRSElapsedTime'
]

categorical_features = [
    'DayOfWeek', 'Month', 'Quarter',
    'OriginAirportID',
    'DestAirportID'
]

numeric_transformer = Pipeline(steps=[
    ('log1p_distance', ColumnTransformer([
        ('log', FunctionTransformer(np.log1p), ['Distance', 'wind_gusts_10m_dest_dep_time', 'wind_gusts_10m'])
    ], remainder='passthrough', verbose_feature_names_out=False)), 
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('OHE', ColumnTransformer([
        ('top50', OneHotEncoder(handle_unknown='ignore', sparse_output=False, max_categories=11), ['OriginAirportID', 'DestAirportID']),
    ]))
    
])

preprocessor = Pipeline([
    ('initial_logger', ProgressLogger(total_rows=13000000, name='Starting Pipeline')),
    
    ('delay_trend', DelayTrendEncoder(
        date_col='dep_datetime',
        origin_col='OriginAirportID',
        dest_col='DestAirportID',
        status_col='Flight_Status',
        output_col='Delay_Trend',
        top_n_airports=15
    )),
    
    ('trend_logger', ProgressLogger(total_rows=13000000, name='Delay Trend Complete')),
    
    ('tail_reuse', SameDayTailReuseEncoder(
        datetime_col='dep_datetime',
        tail_col='Tail_Number',
        output_col='Same_Day_Tail_Reuse'
    )),
    
    ('reuse_logger', ProgressLogger(total_rows=13000000, name='Tail Reuse Complete')),
    
    ('turnaround_delay', TurnaroundDelayEncoder(
        datetime_col='dep_datetime',
        dep_time_col='CRSDepTime',
        arr_time_col='CRSArrTime',
        tail_col='Tail_Number',
        output_prefix=''
    )),
    
    ('turnaround_logger', ProgressLogger(total_rows=13000000, name='Turnaround Delay Complete')),
    
    ('slack_time', SlackTimeEncoder(
        arr_col='CRSArrTime',
        elapsed_col='CRSElapsedTime',
        output_col='Slack_Time'
    )),
    
    ('slack_logger', ProgressLogger(total_rows=13000000, name='Slack Time Complete')),
    
    ('feature_prep', ColumnTransformer([
        ('numeric', numeric_transformer, numerical_features),
        ('categorical', categorical_transformer, categorical_features),
        ('drop', 'drop', [
            'Aircraft_Model', 'Aircraft_EngineType', 'Holiday',
            'dep_datetime', 'Tail_Number', 'temperature_120m_dest_dep_time', 
            'temperature_180m_dest_dep_time', 'temperature_80m_dest_dep_time', 'temperature_180m', 
            'temperature_120m', 'temperature_80m', 'wind_speed_80m', 'wind_speed_120m',
            'wind_speed_180m', 'wind_direction_80m', 'wind_direction_120m', 'wind_direction_180m',
            'wind_speed_80m_dest_dep_time', 'wind_speed_120m_dest_dep_time', 'wind_speed_180m_dest_dep_time',
            'wind_direction_80m_dest_dep_time', 'wind_direction_120m_dest_dep_time', 'wind_direction_180m_dest_dep_time', 
            'soil_temperature_0cm_dest_dep_time', 'soil_temperature_0cm', 'rain', 'rain_dest_dep_time', 'dew_point_2m_dest_dep_time', 
            'cloud_cover_mid', 'cloud_cover', 'cloud_cover_mid_dest_dep_time', 'cloud_cover_dest_dep_time', 'dew_point_2m', 'wind_speed_10m', 
            'wind_speed_10m_dest_dep_time', 'visibility_dest_dep_time', 'visibility', 'apparent_temperature', 'apparent_temperature_dest_dep_time',
            'wind_speed_10m', 'wind_speed_10m_dest_dep_time', 'visibility_dest_dep_time', 'visibility', 'apparent_temperature', 
            'apparent_temperature_dest_dep_time', 'vapour_pressure_deficit', 'vapour_pressure_deficit_dest_dep_time', 'Flights',
            'DOT_ID_Operating_Airline', 'DayOfWeek', 'Month', 'Quarter', 'Flight_Number_Operating_Airline'
        ])
    ], remainder='passthrough', verbose_feature_names_out=False, n_jobs=16)),
    
    ('prep_logger', ProgressLogger(total_rows=13000000, name='Feature Prep Complete')),
    ('select', PCA(
        n_components=50,
        random_state=42,
    )),
    ('final_logger', ProgressLogger(total_rows=13000000, name='Pipeline Complete'))
])
# so that I dont forget, dep_datetime is dropped because of redundancy

preprocessor

Pipeline(steps=[('initial_logger',
                 ProgressLogger(name='Starting Pipeline', total_rows=13000000)),
                ('delay_trend', DelayTrendEncoder()),
                ('trend_logger',
                 ProgressLogger(name='Delay Trend Complete',
                                total_rows=13000000)),
                ('tail_reuse', SameDayTailReuseEncoder()),
                ('reuse_logger',
                 ProgressLogger(name='Tail Reuse Complete',
                                total_rows=13000000)),
                ('turnaround_delay', T...
                                                   'soil_temperature_0cm',
                                                   'rain', 'rain_dest_dep_time',
                                                   'dew_point_2m_dest_dep_time',
                                                   'cloud_cover_mid',
                                                   'cloud_cover', ...])],
                                   verbose_feature_names_out=False)),
                ('prep_logger',
                 ProgressLogger(name='Feature Prep Complete',
                                total_rows=13000000)),
                ('select', PCA(n_components=50, random_state=42)),
                ('final_logger',
                 ProgressLogger(name='Pipeline Complete',
                                total_rows=13000000))])

In [46]:
# Train the pipeline in this cell and run it on X, this cell must end with the transformed data
X = preprocessor.fit_transform(X_train, y_train)

pd.set_option('display.max_columns', None)
X

Starting Pipeline processing started on 13,000,000 rows
Starting Pipeline: 101.4% complete (13,184,010/13,000,000 rows) - Est. remaining: -0.0s
Delay Trend Complete processing started on 13,000,000 rows
Delay Trend Complete: 101.4% complete (13,184,010/13,000,000 rows) - Est. remaining: -0.0s
Tail Reuse Complete processing started on 13,000,000 rows
Tail Reuse Complete: 101.4% complete (13,184,010/13,000,000 rows) - Est. remaining: -0.0s
Turnaround Delay Complete processing started on 13,000,000 rows
Turnaround Delay Complete: 101.4% complete (13,184,010/13,000,000 rows) - Est. remaining: -0.0s
Slack Time Complete processing started on 13,000,000 rows
Slack Time Complete: 101.4% complete (13,184,010/13,000,000 rows) - Est. remaining: -0.0s
Feature Prep Complete processing started on 13,000,000 rows
Feature Prep Complete: 101.4% complete (13,184,010/13,000,000 rows) - Est. remaining: -0.0s
Pipeline Complete processing started on 13,000,000 rows
Pipeline Complete: 101.4% complete (13,184

,pca0,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,pca11,pca12,pca13,pca14,pca15,pca16,pca17,pca18,pca19,pca20,pca21,pca22,pca23,pca24,pca25,pca26,pca27,pca28,pca29,pca30,pca31,pca32,pca33,pca34,pca35,pca36,pca37,pca38,pca39,pca40,pca41,pca42,pca43,pca44,pca45,pca46,pca47,pca48,pca49
235864,487.025799,1497.809550,-575.266798,-39.177454,-14.076114,6.117696,12.232193,2.048477,1.136109,0.114982,-2.994523,-1.060071,0.155850,0.226831,-1.887979,-0.130119,-0.508262,0.129542,0.179037,0.553205,-0.227497,0.154985,-0.917666,0.229411,-0.637793,-0.011943,0.354021,0.429442,0.682814,-0.454833,0.005914,0.777888,0.003919,0.121829,0.313940,1.016523,0.015951,-1.597177,-0.116456,-0.307074,1.091324,0.263032,0.514377,0.796697,0.401251,0.433580,1.743493,-0.674264,0.366828,0.180656
5669442,1307.676139,3273.959322,506.453329,-73.792215,-12.399890,-6.256579,-14.777565,0.304839,-1.227717,0.562761,1.287973,0.226223,0.029165,0.058792,-0.408980,0.234493,-0.460789,-0.523020,1.078595,-0.750012,-0.005820,-0.419148,1.125685,-0.027675,1.111106,0.468178,-0.613082,-0.889665,0.200988,-0.929824,0.197615,-0.203303,0.721982,-0.123404,0.050435,0.163697,0.009103,-0.235271,0.819520,-0.347281,-0.587041,-0.095511,-0.193617,0.587371,0.753527,-0.219008,0.610648,-0.926399,0.051250,0.023735
11140289,-1322.710408,3279.461309,-85.575792,-42.787814,-12.521943,6.250060,-14.780949,-0.581547,-0.809765,0.064909,-1.003894,-0.262784,0.040684,0.038034,-0.615394,0.481104,-0.293473,0.370824,1.289954,-0.474917,0.137507,0.174062,-0.853756,-0.294711,-0.618684,0.010698,0.219762,0.782113,0.177805,0.661792,0.237793,0.017053,0.235786,1.404430,0.006611,-0.267787,0.018458,-0.147747,-1.225978,0.004400,0.952680,-0.030362,0.107640,0.238176,-0.646439,-0.340319,-0.258284,-0.830055,0.280408,0.053146
1085807,646.798585,2634.267766,847.600012,-85.285296,-12.888568,-6.146110,-13.783399,1.113372,-0.410805,0.531321,1.516511,-0.168890,-0.302091,0.110759,-0.464364,-0.306072,0.000368,-0.654836,-0.266748,0.005938,0.100921,-0.472220,1.307567,0.377796,1.425030,0.042618,-0.910519,-0.889194,0.195782,-0.365976,0.188289,-0.153213,0.266903,0.548589,0.052248,-0.353491,0.000392,-0.606539,0.288677,0.558250,0.417749,-0.104651,0.395903,0.033888,-0.911841,-0.070986,0.268684,-0.761545,-0.045121,0.453496
2762781,-705.467841,2637.964003,296.072273,-87.961928,-13.025424,6.391703,-13.772958,0.361667,-0.035716,0.195875,-1.624701,0.279386,-0.561002,0.050457,-0.198469,0.443121,-0.285841,0.398394,-0.160168,-0.515282,0.176176,-0.011493,-0.137993,0.211181,0.356671,0.508222,-0.277925,0.704519,0.385068,-1.143346,-0.483054,0.665297,-0.368611,0.958199,0.085665,-0.033694,0.077917,-0.224086,-1.281737,1.050714,0.586515,0.012504,0.157895,-0.469306,0.177889,-0.025231,-0.261322,-0.630203,0.551919,0.143153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858254,-127.392557,176.850322,-1133.917884,-63.885157,-11.713171,-10.446447,15.276866,3.518832,-0.476031,0.453078,0.721966,-0.257719,0.625826,-0.232573,1.430988,3.767466,4.247659,-0.249452,-1.694217,-0.688644,-1.061340,-0.521422,2.480138,-1.451152,0.956149,0.102851,-0.386387,-0.447937,0.550387,-1.926958,0.411453,-2.026784,-0.182620,-0.254649,-2.889424,1.274712,0.098696,-1.080367,-0.799591,1.486776,-0.404748,-0.155058,-0.307859,-0.698343,-0.145982,0.141696,-0.701790,-0.389795,-0.560052,-0.305785
7932517,1010.489612,4193.908570,-1098.263067,-29.063981,-2.802458,15.155686,15.257801,2.148074,-1.019992,0.771895,0.784603,-0.737374,0.057966,0.075140,-0.130976,2.463591,5.523600,-0.376507,-0.715865,0.734658,-1.068485,0.404639,-0.027309,-0.616285,-0.070336,0.191119,-0.033028,0.120656,-0.118369,-0.466044,-0.086530,1.019604,0.059486,-0.184103,-2.863342,-0.621589,0.035772,-0.888682,-1.218375,0.652933,2.125565,-0.016578,-0.073727,0.101771,0.085089,-0.004666,-0.600021,-0.511857,0.702953,0.937362
196209,-453.120652,-535.790988,-1210.954843,1.805676,-18.

In [26]:
X.isna().sum().sort_values(ascending=False).head(20)

CRSElapsedTime                              5
Flight_Number_Operating_Airline             1
surface_pressure_dest_dep_time              0
wind_direction_10m                          0
precipitation_dest_dep_time                 0
showers_dest_dep_time                       0
snowfall_dest_dep_time                      0
snow_depth_dest_dep_time                    0
soil_moisture_0_to_1cm_dest_dep_time        0
et0_fao_evapotranspiration_dest_dep_time    0
evapotranspiration_dest_dep_time            0
cloud_cover_high_dest_dep_time              0
cloud_cover_low_dest_dep_time               0
weather_code_dest_dep_time                  0
weather_code                                0
pressure_msl_dest_dep_time                  0
wind_direction_10m_dest_dep_time            0
Year                                        0
DayofMonth                                  0
OriginCityMarketID                          0
dtype: int64

In [75]:
# Save the preprocessor as a parquet file
X.to_parquet("preprocessor.parquet")


In [78]:
# pickle the preprocessor
import pickle
with open("preprocessor.pkl", "wb") as f:
    pickle.dump(preprocessor, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import sys

for name, step in preprocessor.named_steps.items():
    print(name, type(step), sys.getsizeof(step))

initial_logger <class '__main__.ProgressLogger'> 48
delay_trend <class '__main__.DelayTrendEncoder'> 48
trend_logger <class '__main__.ProgressLogger'> 48
tail_reuse <class '__main__.SameDayTailReuseEncoder'> 48
reuse_logger <class '__main__.ProgressLogger'> 48
turnaround_delay <class '__main__.TurnaroundDelayEncoder'> 48
turnaround_logger <class '__main__.ProgressLogger'> 48
slack_time <class '__main__.SlackTimeEncoder'> 48
slack_logger <class '__main__.ProgressLogger'> 48
feature_prep <class 'sklearn.compose._column_transformer.ColumnTransformer'> 48
prep_logger <class '__main__.ProgressLogger'> 48
select <class 'sklearn.feature_selection._from_model.SelectFromModel'> 48
final_logger <class '__main__.ProgressLogger'> 48


# Models

## Decision Tree

In [47]:
from sklearn.metrics import make_scorer, precision_score, recall_score, precision_recall_curve
def precision_at_recall(y, y_pred, *, recall, **kwargs):
    """
    Calculate the precision at a given recall level.

    To use this with cross_val_score, you need to use the make_scorer function to
    create a scoring function that can be used with cross_val_score. For example:

        scorer = make_scorer(precision_at_recall, recall=0.9)  # 0.9 is the minimum recall level
        cross_val_score(model, X, y, cv=5, scoring=scorer)

    The default will only work for binary classification problems. You must change the
    average parameter if you want to use for multiclass classification. For example:

        scorer = make_scorer(precision_at_recall, recall=0.9, average="micro")
    """
    return precision_score(y, y_pred, **kwargs) if recall_score(y, y_pred, **kwargs) > recall else 0.0

def recall_at_precision(y, y_pred, *, precision, **kwargs):
    """
    Calculate the recall at a given precision level.

    To use this with cross_val_score, you need to use the make_scorer function to
    create a scoring function that can be used with cross_val_score. For example:

        scorer = make_scorer(recall_at_precision, precision=0.9)
        cross_val_score(model, X, y, cv=5, scoring=scorer)

    The default will only work for binary classification problems. You must change the
    average parameter if you want to use for multiclass classification. For example:

        scorer = make_scorer(recall_at_precision, precision=0.9, average="micro")
    """
    return recall_score(y, y_pred, **kwargs) if precision_score(y, y_pred, **kwargs) > precision else 0.0

In [65]:
X_sample = X.sample(n=100000, random_state=42)
y_train = y_train.sample(n=100000, random_state=42)

In [69]:
from sklearn.tree import DecisionTreeClassifier

# Create a DecisionTreeClassifier
dt = DecisionTreeClassifier(
    random_state=42,
    max_depth=20,
    min_samples_leaf=10,
    class_weight='balanced'
)

scorer = make_scorer(precision_at_recall, recall=0.25, average='weighted')

scores = cross_val_score(dt, X_sample, y_train, cv=5, scoring=scorer, verbose=1, n_jobs=64)

# Fit the model and predict using cross-validation
y_pred_cv = cross_val_predict(dt, X_sample, y_train, cv=5, n_jobs=64)

# Calculate recall, precision, and F1 score
recall = recall_score(y_train, y_pred_cv, average='macro')
precision = precision_score(y_train, y_pred_cv, average='macro', zero_division=0)
f1 = f1_score(y_train, y_pred_cv, average='macro')

print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")

# Cross-validated performance
print("Cross-Validation Classification Report:")
print(classification_report(y_train, y_pred_cv, zero_division=0))

print(f"Mean precision at recall 0.25: {scores.mean():.4f} ± {scores.std():.4f}")


[Parallel(n_jobs=64)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=64)]: Done   5 out of   5 | elapsed:   40.4s finished


Recall: 0.0565
Precision: 0.0547
F1 Score: 0.0130
Cross-Validation Classification Report:
                                  precision    recall  f1-score   support

            Carrier Cancellation       0.00      0.09      0.01       424
      Major Delay - CarrierDelay       0.00      0.05      0.01       514
 Major Delay - LateAircraftDelay       0.01      0.09      0.01       649
          Major Delay - NASDelay       0.00      0.02      0.00        91
      Major Delay - WeatherDelay       0.00      0.02      0.00        96
     Medium Delay - CarrierDelay       0.02      0.03      0.02      1881
Medium Delay - LateAircraftDelay       0.02      0.02      0.02      2886
         Medium Delay - NASDelay       0.01      0.08      0.01       487
     Medium Delay - WeatherDelay       0.00      0.06      0.01       325
      Minor Delay - CarrierDelay       0.04      0.02      0.03      3539
 Minor Delay - LateAircraftDelay       0.03      0.02      0.02      4238
          Minor Delay

In [68]:
y_train.value_counts()

Flight_Status
On-Time                             78470
Minor Delay - LateAircraftDelay      4238
Unknown                              3977
Minor Delay - CarrierDelay           3539
Medium Delay - LateAircraftDelay     2886
Medium Delay - CarrierDelay          1881
Minor Delay - NASDelay               1060
Weather Cancellation                  781
Major Delay - LateAircraftDelay       649
Major Delay - CarrierDelay            514
Medium Delay - NASDelay               487
Carrier Cancellation                  424
Minor Delay - WeatherDelay            326
Medium Delay - WeatherDelay           325
NAS Cancellation                      205
Major Delay - WeatherDelay             96
Major Delay - NASDelay                 91
Security Issue                         51
Name: count, dtype: int64

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score, cross_val_predict

# Create Gradient Boosting classifier
model = GradientBoostingClassifier(
    n_estimators=10,    # Number of trees
    max_depth=3,         # Max tree depth
    learning_rate=0.1,   # Slightly lower learning rate for better stability
    subsample=0.8,       # Use 80% of samples for each tree (helps with large datasets)
    random_state=42,
    verbose=2            # Show progress
)

# Create scorer for precision at recall
scorer = make_scorer(precision_at_recall, recall=0.25, average='weighted')

# Run cross-validation with the Gradient Boosting model
scores = cross_val_score(model, X_sample, y_train, cv=5, scoring=scorer, verbose=1, n_jobs=16)

# Fit the model and predict using cross-validation
y_pred_cv = cross_val_predict(model, X_sample, y_train, cv=5, n_jobs=16)

# Calculate metrics
recall = recall_score(y_train, y_pred_cv, average='macro')
precision = precision_score(y_train, y_pred_cv, average='macro', zero_division=0)
f1 = f1_score(y_train, y_pred_cv, average='macro')

print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")

# Print classification report
print("Cross-Validation Classification Report:")
print(classification_report(y_train, y_pred_cv, zero_division=0))
print(f"Mean precision at recall 0.25: {scores.mean():.4f} ± {scores.std():.4f}")


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.


In [8]:
classes = np.unique(y_train)
y_train_bin = label_binarize(y_train, classes=classes)

y_scores = cross_val_predict(dt_clf, X, y_train, cv=cv, method='predict_proba')

plt.figure(figsize=(10, 6))
for i, class_name in enumerate(classes):
    precision, recall, _ = precision_recall_curve(y_train_bin[:, i], y_scores[:, i])
    plt.plot(recall, precision, lw=2, label=f"{class_name}")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curves (One-vs-Rest)")
plt.legend(loc="best", fontsize='small')
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
for i, class_name in enumerate(classes):
    fpr, tpr, _ = roc_curve(y_train_bin[:, i], y_scores[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f"{class_name} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves (One-vs-Rest)")
plt.legend(loc="lower right", fontsize='small')
plt.grid(True)
plt.tight_layout()
plt.show()

KeyboardInterrupt: 

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report
)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

log_reg = LogisticRegression(
    class_weight='balanced',
    solver='lbfgs',
    max_iter=2000,  
    random_state=42
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred_cv_log_reg = cross_val_predict(log_reg, X_scaled, y_train, cv=cv)

log_reg.fit(X_scaled, y_train)
y_pred_train_log_reg = log_reg.predict(X_scaled)

print("Performance on Training Set:")
print(f"Accuracy: {accuracy_score(y_train, y_pred_train_log_reg):.4f}")
print(f"Precision: {precision_score(y_train, y_pred_train_log_reg, average='macro', zero_division=0):.4f}")
print(f"Recall: {recall_score(y_train, y_pred_train_log_reg, average='macro'):.4f}")
print(f"F1 Score: {f1_score(y_train, y_pred_train_log_reg, average='macro'):.4f}\n")

print("Cross-Validated Performance:")
print(f"CV Accuracy: {accuracy_score(y_train, y_pred_cv_log_reg):.4f}")
print(f"CV Precision: {precision_score(y_train, y_pred_cv_log_reg, average='macro', zero_division=0):.4f}")
print(f"CV Recall: {recall_score(y_train, y_pred_cv_log_reg, average='macro'):.4f}")
print(f"CV F1 Score: {f1_score(y_train, y_pred_cv_log_reg, average='macro'):.4f}\n")

print("Cross-Validation Classification Report:")
print(classification_report(y_train, y_pred_cv_log_reg, zero_division=0))

KeyboardInterrupt: 

In [ ]:
# use MLP classifier on the pipeline

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, precision_score, f1_score

X_test = test_data.drop('Flight_Status', axis=1)
y_test = test_data['Flight_Status']

# Your X_prepped is already available from previous preprocessing
# Now we'll create and train the MLP classifier optimized for precision

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, precision_score

# Create the MLP classifier with precision-focused parameters
mlp = MLPClassifier(
    hidden_layer_sizes=(100,), 
    random_state=42            
)

# Train the model using your preprocessed X data
mlp.fit(X_prepped, y)

# For testing, you'll need to apply the same preprocessing to X_test
# If you haven't already done this, you would typically do:
# X_test_prepped = preprocessor.transform(X_test)
# But since you didn't specify this step, I'll assume you need to do it:

# Make predictions (using whatever preprocessed test data you have)
# If X_test is already preprocessed:
y_pred = mlp.predict(X_test)
# If you need to preprocess X_test:
# y_pred = mlp.predict(X_test_prepped)

# Evaluate with focus on precision
print("Precision score:", precision_score(y_test, y_pred, average='macro'))
print("F1 score:", f1_score(y_test, y_pred, average='macro'))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''